# Analiza wypływu parametrów na algorytm mrówkowy

In [19]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import re
from pathlib import Path
from glob import glob
from collections import defaultdict

Przydatne funkcje

In [24]:
def load_mrowczenie(file_path: str) -> defaultdict:
    line_pattern = r'Pokolenie nr=\s*(?P<generation>\d+)\s+najlepszy=\s*(?P<best_ant_num>\d+)\(dl=\s*(?P<best_value>[-+]?[0-9]*\.?[0-9]*)\),\s*najgorszy=\s*(?P<worst_ant_num>\d+)\(dl=\s*(?P<worst_value>[-+]?[0-9]*\.?[0-9]*)\),\s*średnia_dl=\s*(?P<mean_value>[-+]?[0-9]*\.?[0-9]*)'
    
    with open(file_path, 'r', encoding='windows-1250') as file:
        lines = file.readlines()
        
#     data = []
    data = defaultdict(list)
    for line in lines:
        match = re.match(line_pattern, line)
        if not match:
            print('Line didn\'t match!')
            continue
            
        data['generacja'].append(match.group('generation'))
        data['najlepsza mrówka'].append(match.group('best_ant_num'))
        data['najlepsza wartość'].append(match.group('best_value'))
        data['najgorsza mrówka'].append(match.group('worst_ant_num'))
        data['najgorsza wartość'].append(match.group('worst_value'))
        data['średnia wartość'].append(match.group('mean_value'))
#         data.append([match.group(g) for g in ['generation', 'best_ant_num', 'best_value', 'worst_ant_num', 'worst_value', 'mean_value']])
        
    return data

In [53]:
def load_parameter_messurements(directory: str) -> pd.DataFrame:
    directory = Path(directory)
    parameter_name = directory.stem
    parameter_dirs = sorted(d for d in directory.iterdir() if d.is_dir())
    
    data = defaultdict(list)
    for parameter_dir in parameter_dirs:
        parameter_value = parameter_dir.name
        parameter_data = load_mrowczenie(parameter_dir / Path('mrowczenie.txt'))
        for k, v in parameter_data.items():
            data[k].extend(v)
        n = len(parameter_data[list(parameter_data.keys())[0]])
        data['testowany parametr'].extend([parameter_name] * n)
        data['wartość parametru'].extend([parameter_value] * n)
        
    df = pd.DataFrame(data)
    df = df.astype({
        'generacja': int,
        'najlepsza mrówka': int,
        'najlepsza wartość': float,
        'najgorsza mrówka': int,
        'najgorsza wartość': float,
        'średnia wartość': float,
        'wartość parametru': float
    })
    return df

In [218]:
def plot_single_parameters(df, name) -> go.Figure:
    fig = go.Figure(
        layout=dict(
            xaxis=dict(title='Pokolenie'),
            yaxis=dict(title='Wartość dl'),
            title=f'Średnia ruchoma parametrów dla kolejnych pokoleń {name}',
            margin=dict(t=50, b=0, r=30),
            legend_orientation='h'
        ),
        data=[
            go.Scatter(x=df.generacja, y=df['najlepsza wartość'], name='najlepsza wartość'),
            go.Scatter(x=df.generacja, y=df['najgorsza wartość'].rolling(10, min_periods=5).mean(), name='najgorsza wartość'),
            go.Scatter(x=df.generacja, y=df['średnia wartość'].rolling(5).mean(), name='średnia'),
        ]
    )
    return fig

## aco_ro

In [237]:
df_aco_ro = load_parameter_messurements('wyniki/aco_ro')
df_aco_ro

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość,testowany parametr,wartość parametru
0,0,0,17.689845,38,23.121257,19.466476,aco_ro,0.001
1,1,22,18.664647,45,21.463621,19.287995,aco_ro,0.001
2,2,4,17.429772,20,21.104134,19.319427,aco_ro,0.001
3,3,0,17.429772,47,21.155713,19.450503,aco_ro,0.001
4,4,0,17.429772,17,22.096625,19.404355,aco_ro,0.001
...,...,...,...,...,...,...,...,...
3995,995,0,17.153895,30,20.426304,19.708882,aco_ro,0.070
3996,996,0,17.153895,21,21.187873,19.881938,aco_ro,0.070
3997,997,0,17.153895,5,21.187873,19.731635,aco_ro,0.070
3998,998,0,17.153895,15,21.187873,19.637718,aco_ro,0.070


### 0.001

In [238]:
df_aco_ro_001 = df_aco_ro.query('`wartość parametru` == 0.001').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_ro_001
name = 'aco_ro = 0.001'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
0,0,0,17.689845,38,23.121257,19.466476
1,1,22,18.664647,45,21.463621,19.287995
2,2,4,17.429772,20,21.104134,19.319427
3,3,0,17.429772,47,21.155713,19.450503
4,4,0,17.429772,17,22.096625,19.404355
...,...,...,...,...,...,...
995,995,0,17.259699,31,20.311431,19.189281
996,996,0,17.259699,7,23.992306,19.483301
997,997,0,17.259699,32,20.311431,19.035494
998,998,0,17.259699,10,20.201967,19.375485


In [240]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_ro = 0.001'
})


### 0.01

In [241]:
df_aco_ro_01 = df_aco_ro.query('`wartość parametru` == 0.01').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_ro_01
name = 'aco_ro = 0.01'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
1000,0,48,16.544032,8,22.189296,19.214272
1001,1,0,16.544032,33,21.698865,19.594696
1002,2,8,16.780851,18,21.828245,19.360591
1003,3,0,16.780851,44,22.000488,19.533207
1004,4,0,16.780851,17,19.585089,19.306211
...,...,...,...,...,...,...
1995,995,0,16.669637,47,20.583294,18.809256
1996,996,0,16.669637,17,18.973309,18.692418
1997,997,0,16.669637,41,19.816390,18.548957
1998,998,0,16.669637,26,20.093224,18.670953


In [242]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_ro = 0.01'
})


### 0.02

In [243]:
df_aco_ro_02 = df_aco_ro.query('`wartość parametru` == 0.02').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_ro_02
name = 'aco_ro = 0.02'

df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
2000,0,0,17.689845,8,22.189296,19.527035
2001,1,10,16.845059,29,23.206053,19.472467
2002,2,0,16.845059,35,21.954899,19.357145
2003,3,17,16.509320,39,22.096625,19.550688
2004,4,0,16.509320,5,20.839620,19.362392
...,...,...,...,...,...,...
2995,995,0,16.552583,45,20.629336,18.618597
2996,996,0,16.552583,32,20.643967,18.835570
2997,997,0,16.552583,29,19.330124,18.929798
2998,998,0,16.552583,11,20.478314,18.986046


In [244]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_ro = 0.02'
})


### 0.07

In [246]:
df_aco_ro_07 = df_aco_ro.query('`wartość parametru` == 0.07').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_ro_07
name = 'aco_ro = 0.07'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
3000,0,0,17.689845,31,21.879786,19.501309
3001,1,0,17.689845,40,21.407642,19.494288
3002,2,0,17.689845,16,24.516883,19.697890
3003,3,0,17.689845,21,21.594254,19.545717
3004,4,0,17.689845,43,21.189761,19.766873
...,...,...,...,...,...,...
3995,995,0,17.153895,30,20.426304,19.708882
3996,996,0,17.153895,21,21.187873,19.881938
3997,997,0,17.153895,5,21.187873,19.731635
3998,998,0,17.153895,15,21.187873,19.637718


In [248]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_ro = 0.07'
})


### Porównanie

In [236]:
fig = px.line(df_aco_ro, x='generacja', y='najlepsza wartość', color='wartość parametru',
             title='Wartość najlepszego osobnika w zależności od aco_ro')
fig.update_yaxes(title_text='Wartość dl')
fig.update_xaxes(title_text='Pokolenie')
fig.update_layout(margin=dict(t=50, b=0, r=30), legend_orientation='v')

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Wartość najlepszego osobnika w zależności od aco_ro'
})


In [230]:
fig = px.line(df_aco_ro, x='generacja', y=df_aco_ro['średnia wartość'].rolling(5).mean(), 
              color='wartość parametru',
              title='Średnia wartość w zależności od aco_ro')
fig.update_yaxes(title_text='Wartość dl')
fig.update_xaxes(title_text='Pokolenie')
fig.update_layout(margin=dict(t=50, b=0, r=30), legend_orientation='v')

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia wartość w zależności od aco_ro'
})


## aco_alfa

In [231]:
df_aco_alfa = load_parameter_messurements('wyniki/aco_alfa')
df_aco_alfa

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość,testowany parametr,wartość parametru
0,0,45,16.750883,49,21.968958,19.604503,aco_alfa,0.5
1,1,0,16.750883,8,23.613458,19.799045,aco_alfa,0.5
2,2,0,16.750883,16,21.257485,19.575276,aco_alfa,0.5
3,3,47,17.369803,34,22.419693,19.596351,aco_alfa,0.5
4,4,0,17.369803,41,21.898665,19.749420,aco_alfa,0.5
...,...,...,...,...,...,...,...,...
3995,995,0,17.654196,47,19.589807,18.676697,aco_alfa,2.0
3996,996,0,17.654196,11,19.413171,18.660359,aco_alfa,2.0
3997,997,0,17.654196,5,21.739988,18.610928,aco_alfa,2.0
3998,998,0,17.654196,41,19.589807,18.380930,aco_alfa,2.0


### 0.5

In [175]:
df_aco_alfa_05 = df_aco_alfa[df_aco_alfa['wartość parametru'] == 0.5].drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_alfa_05
name = 'aco_alfa = 0.5'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
0,0,45,16.750883,49,21.968958,19.604503
1,1,0,16.750883,8,23.613458,19.799045
2,2,0,16.750883,16,21.257485,19.575276
3,3,47,17.369803,34,22.419693,19.596351
4,4,0,17.369803,41,21.898665,19.749420
...,...,...,...,...,...,...
995,995,0,19.127595,22,20.905448,19.131230
996,996,0,19.127595,46,23.500579,19.585200
997,997,0,19.127595,33,23.500579,19.546505
998,998,0,19.127595,5,22.171106,19.241195


In [176]:
fig = px.line(df, x="generacja", y="średnia wartość", title='Średnia wartość')
fig.update_layout(margin=dict(t=50, b=0))

fig.show()
fig.write_image('fig.svg')

In [201]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_alfa = 0.5'
})


### 1.0

In [202]:
df_aco_alfa_10 = df_aco_alfa.query('`wartość parametru` == 1.0').drop(['testowany parametr', 'wartość parametru'], axis=1)
name = 'aco_alfa = 1.0'
df = df_aco_alfa_10
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
1000,0,0,17.689845,13,21.632799,19.641927
1001,1,0,17.689845,39,21.407642,19.642451
1002,2,0,17.689845,32,22.434226,19.515814
1003,3,0,17.689845,27,22.351598,19.483345
1004,4,7,17.104989,18,21.916669,19.431419
...,...,...,...,...,...,...
1995,995,0,16.721228,23,20.802080,18.892829
1996,996,0,16.721228,30,21.779255,19.259180
1997,997,0,16.721228,23,19.864803,19.425622
1998,998,0,16.721228,2,19.016135,19.106130


In [205]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_alfa = 1.0'
})


### 1.5

In [207]:
df_aco_alfa_15 = df_aco_alfa.query('`wartość parametru` == 1.5').drop(['testowany parametr', 'wartość parametru'], axis=1)
name = 'aco_alfa = 1.5'
df = df_aco_alfa_15
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
2000,0,48,16.544032,8,22.189296,19.214272
2001,1,0,16.544032,33,21.698865,19.594696
2002,2,8,16.780851,18,21.828245,19.360591
2003,3,0,16.780851,44,22.000488,19.533207
2004,4,0,16.780851,17,19.585089,19.306211
...,...,...,...,...,...,...
2995,995,0,16.669637,47,20.583294,18.809256
2996,996,0,16.669637,17,18.973309,18.692418
2997,997,0,16.669637,41,19.816390,18.548957
2998,998,0,16.669637,26,20.093224,18.670953


In [209]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_alfa = 1.5'
})


### 2.0

In [210]:
df_aco_alfa_20 = df_aco_alfa.query('`wartość parametru` == 2.0').drop(['testowany parametr', 'wartość parametru'], axis=1)
name = 'aco_alfa = 2.0'
df = df_aco_alfa_20
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
3000,0,0,17.689845,37,21.668315,19.711217
3001,1,0,17.689845,30,21.283011,19.322335
3002,2,2,18.196735,38,22.000488,19.456536
3003,3,35,16.877477,26,21.690811,19.571461
3004,4,0,16.877477,38,20.845922,19.273594
...,...,...,...,...,...,...
3995,995,0,17.654196,47,19.589807,18.676697
3996,996,0,17.654196,11,19.413171,18.660359
3997,997,0,17.654196,5,21.739988,18.610928
3998,998,0,17.654196,41,19.589807,18.380930


In [213]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_alfa = 2.0'
})


### Porównanie

In [222]:
fig = px.line(df_aco_alfa, x='generacja', y='najlepsza wartość', color='wartość parametru',
             title='Wartość najlepszego osobnika w zależności od aco_alfa')
fig.update_yaxes(title_text='Wartość dl')
fig.update_xaxes(title_text='Pokolenie')
fig.update_layout(margin=dict(t=50, b=0, r=30), legend_orientation='v')

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Wartość najlepszego osobnika w zależności od aco_alfa'
})


In [223]:
fig = px.line(df_aco_alfa, x='generacja', y=df_aco_alfa['średnia wartość'].rolling(5).mean(), 
              color='wartość parametru',
              title='Średnia wartość w zależności od aco_alfa')
fig.update_yaxes(title_text='Wartość dl')
fig.update_xaxes(title_text='Pokolenie')
fig.update_layout(margin=dict(t=50, b=0, r=30), legend_orientation='v')

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia wartość w zależności od aco_alfa'
})
